In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")

Running on cuda


In [3]:
import whisper
model = whisper.load_model("large", download_root="models").to(device=device) # load "large" model

In [4]:
import os
import pandas as pd
import time
import json

In [5]:
done = [
    "A.R.O.G _ Cem Yılmaz Türk Komedi Filmi.mp3",
    "Abimm  _ Türk Filmi Tek Parça (HD).mp3",
    "Acıların Kadını Bergen _ Yerli Film _ Weco Film.mp3",
    "Allah'ın Evi Filmi  _ Türk Dram Filmi   _ Yerli Film Tek Parça (2023).mp3",
    "Arekalar ve Cinler _ Türk Korku Filmi  _ 4K UHD  _ Full İzle.mp3",
    "Artık Çok Geç _ TV Filmi Full (Fikret Kuşkan, Canan Mutluer, Ali Atay).mp3",
    "Aşk Bu Mu Film (Afra Saraçoğlu  Kubilay Aka) Tek Parça _ HD.mp3",
    "AURA (Sansürsüz) _ Sinema Filmi (Gani Rüzgar Şavata).mp3",
    "Av Mevsimi _ [4K] Cem Yılmaz Türk Filmi Full İzle.mp3",
    "Ay Lav Yu Tek Parça Film Yerli Film.mp3",
    "Azap _ 2015 _ Yerli Korku Filmi _ Weco Film.mp3",
    "Aşk Olsun _ Türk Komedi Filmi Tek Parça (HD).mp3",
    "Aşk Yolunda _ Yerli Romantik Komedi Filmi Full Hd İzle.mp3",
    "Aşkopat _ Türk Komedi Filmi Tek Parça (HD).mp3",
    "Bana Git De _ Türk Dram Filmi.mp3",
    "Bayrampaşa Ben Fazla Kalmayacağım _ Full Film.mp3",
    "Başka Semtin Çocukları HD.mp3",
    "Behzat Ç  Ankara Yanıyor _ Türk Filmi Tek Parça (HD).mp3",
    "Beyaz Melek HD (2007) _ Türk Filmi.mp3",
    "Beyza'nın Kadınları _ 2005 Tek Parça (Tamer Karadağlı  Demet Evgar).mp3",
    "Bi O Kalmıştı (BOK) _ Türk Filmi Tek Parça (HD).mp3",
    "Bizans Oyunları _ Tek Parça Film (Yerli Komedi) Avşar Film.mp3",
    "Bizim Aile _ FULL HD.mp3",
    "Bizim Hikaye _ Sinema Filmi _ 2015.mp3",
    "Bu Son Olsun _ Türk Dram Filmi _ Full İzle.mp3",
    "Bu İşte Bir Yalnızlık Var _ Türk Filmi Tek Parça (HD).mp3",
    "büyü _ türk korku filmi (2004 hd).mp3",
    "Cem Yılmaz Son filmi izle _ Film izle _ Yerli Komedi filmi izle.mp3",
    "Cin Padişahı 'Karantina' Adlı Yerli Korku Filmi.mp3",
    "DABBE 2 _ FULL Film.mp3",
    "Dabbe 3 (KORKU) Bir Cin Vakası.mp3",
    "Dabbe 4_ Cin Çarpması _ Fragman (192 kbps).mp3",
    "dabbe 5 zehri cin full hd izle.mp3",
    "DABBE _ FULL Film.mp3",
    "Darbe Full İzle Tek Parça İzle 2015 Yapımı.mp3",
    "Davaro Türk Filmi _ FULL _ 4K ULTRA HD _ KEMAL SUNAL _ ŞENER ŞEN.mp3",
    "Dağ 2 full hd izle 1080p.mp3",
    "Dedemin İnsanları _ Tek Parça HD.mp3",
    "Deli Dumrul Kurtlar Kuşlar Aleminde _ Tek Parça Film (Yerli Film).mp3",
    "Devlet Kuşu _ Kemal Sunal Komedi Filmi.mp3",
    "Dünyanın En İyi Dram Filmi Full HD.mp3",
    "Düttürü Dünya (1988) _ Türk Filmi (Kemal Sunal).mp3",
    "Efsunlu _ Kabirden Gelen _ Türk Korku Filmi.mp3",
    "Engelli Aşk _ Sinema Filmi _ 4k Ultra HD.mp3",
    "Entel Koy Efe Koye Karsi 2012.mp3",
    "Eskici Ve Oğulları _ Eski Türk Filmi Tek Parça.mp3",
    "Ev _ Türk Dram Filmi.mp3",
    "Hababam Sınıfı Uyanıyor _ FULL HD.mp3",
    "Hababam Sınıfı Merhaba _ Türk Filmi Tek Parça (HD).mp3",
    "Hababam Sınıfı Güle Güle _ FULL HD.mp3",
    "Hababam Sınıfı Dokuz Doğuruyor _ FULL HD.mp3",
    "Hababam Sınıfı Askerde _ Şafak Sezer Türk Komedi Filmi Tek Parça (HD).mp3",
    "Güneşi Gördüm (2009 _ HD) _ Türk Filmi.mp3",
    "Gülizar _ TV Filmi Full  (Tuba Büyüküstün, Şevket Çoruh).mp3",
    "Gönlümdeki Köşk Olmasa _ Türk Dram Filmi.mp3",
    "Gurbetçi Şaban _ Kemal Sunal Eski Türk Komedi Filmi.mp3",
    "GDO Karakedi _ Türk Komedi Filmi Tek Parça (HD).mp3",
    "GAZAP ( Korku Filmi ) Full ( 4K ).mp3",
    "Feride _ Türk Komedi Filmi _ Tek Parça.mp3",
    "Feride _ Eski Türk Filmi Tek Parça.mp3",
    "Eşkıya _ Şener Şen 4K Dram Filmi.mp3",
    "Hayalet Dayı _ Türkçe Komedi 4K _ Tiwi.mp3",
    "Hababam Sınıfı Üç Buçuk _ Şafak Sezer Türk Komedi Filmi Tek Parça (HD).mp3",
    "Hababam Sınıfı _ FULL HD.mp3",
    "Hayattan Korkma _ Türk Filmi Tek Parça (HD).mp3",
    "Helak Kayıp Köy _ Türk Filmi Tek Parça (HD).mp3",
    "Hemşo _ Tek Parça Film (Yerli Komedi).mp3",
    "Hep Yek _ Türk Komedi Filmi.mp3",
    "Her Şey Çok Güzel Olacak _ Cem Yılmaz _ Mazhar Alanson 4K Komedi Filmi.mp3",
    "Herşey aşktan tek parça film ( yerli film ).mp3",
    "Horoz Bayram  _ Eğlence Dolu Türk Komedi Filmi.mp3",
    "Issız Adam _ Full HD (Yerli Film).mp3",
    "İsyan Filmi Tek Parça _ 1979.mp3",
    "Kapıcılar Kralı Türk Filmi _ 4K ULTRA HD _ KEMAL SUNAL.mp3",
    "Kar Kırmızı _ Yerli Dram Filmi _ Mehmet Usta _ Ahmet Mümtaz Taylan _ Özge Gürel.mp3",
    "Hızlı Adımlar _ TV Filmi Full (Altan Erkekli, Tuba Ünsal, Ezgi Mola).mp3",
    "Kayıp Cennet İnsanları _ Eski Türk Filmi Tek Parça.mp3",
    "Keloğlan ile Cankız _ HD Türk Filmi.mp3",
    "KELOĞLAN _ HD Türk Filmi.mp3",
    "Kendime İyi Bak _ Türk Filmi Tek Parça (HD).mp3",
    "Kim Daha Mutlu _ Yerli Film (Romantik Komedi) Tek Parça _ HD.mp3",
    "Kimse Bizi Konuşmasın _Ödüllü Yerli Film_.mp3",
    "Koca Bulma Sanatı  Romantik Komedi  (2023 FULL HD).mp3",
    "Kolpaçino _ Şafak Sezer Türk Komedi Filmi Tek Parça (HD).mp3",
    "Koğuştaki Mucize.mp3",
    "Kubilay _ 2010 _ Yerli Dram _ Savaş Filmi _ Arda Kural Filmi _ Film Şimdi.mp3",
    "Karpuz Kabuğundan Gemiler Yapmak _ Eski Türk Filmi Tek Parça.mp3",
    "Mahlûkat (2022) _4K_ _ English Subtitles.mp3",
    "Leblebi Tozu _ Türk Komedi Filmi Tek Parça (HD).mp3",
    "Kızım İçin _ Tek Parça Film (Yerli Film) _ Eda Ece.mp3",
    "Kızım Gibi Kokuyorsun _ Yerli Dram Filmi _ Tek Parça.mp3",
    "Kılıbık _ Eski Türk Filmi Tek Parça (Kemal Sunal).mp3",
    "Kılıbık _ Eski Türk Filmi Tek Parça (Kemal Sunal).mp3",
    "KÜÇÜK ORTAK  YERLİ ROMANTİK KOMEDİ FİLMİ   ( DİZİ FİLM İZLEMECE ).mp3",
    "Köçek (2000).mp3",
    "Köyden İndim Şehire _ FULL HD.mp3",
    "Köy Düğünü Yerli Komedi (2023 FULL HD).mp3",
    "Kutsal Damacana _ Türk Filmi Tek Parça (HD).mp3",
    "Kutsal Damacana 2 İtmen _ Şafak Sezer Türk Komedi Filmi _ Full Film İzle (HD).mp3",
    "KUSURSUZ SANSÜRSÜZ türk filmi.mp3",
    "Malkoçoğlu ve Cem Sultan _ FULL HD.mp3",
    "Mandıra Filozofu _ Full Film.mp3",
    "Mandıra Filozofu İstanbul _ Full Film.mp3",
    "Meteler _ Türk Filmi (2019).mp3",
    "MEVSİM ÇİÇEK AÇTI FULL İZLE.mp3",
    "Meryem _ Yerli Dram Filmi _ Full HD Tek Parça.mp3",
    "Memleket _ Tek Parça Film (Yerli Film).mp3",
    "Memleket Meselesi  I 2010 Full Film.mp3",
    "Mel_Un _ Türk Korku Filmi Tek Parça (HD).mp3",
    "Maskeli Beşler Kıbrıs _ Şafak Sezer Türk Komedi Filmi Tek Parça (HD).mp3",
    "Maskeler De Düşer _ Müfit Can Saçıntı’nın Yeni Filmi _ English Subtitle.mp3",
    "Muhallebicinin Oğlu _ TV Filmi Full (Fikret Kuşkan, Yasemin Alkaya, Binnur Kaya).mp3",
    "Musabbar _ Türk Korku Filmi _ 4K UHD.mp3",
    "Muhalif Başkan _ Türk Filmi Tek Parça (HD).mp3",
    "Mucize (2015_HD) _ Türk Filmi.mp3",
    "Mommo Kız Kardeşim _ Yerli Dram Filmi _ Full HD Tek Parça.mp3",
    "Misafir _ Ödüllü Yerli Dram Filmi _ Full HD Tek Parça.mp3",
    "Mezarlık _ Türk Korku Filmi.mp3",
    "Musallat _ Türkçe Dublaj _ Yerli Gerilim Korku Filmi Full HD İzle.mp3",
    "Mutluluk  Murat Han    Özgü Namal.mp3",
    "Namuslu _ Türk Filmi (Restorasyonlu) _ Şener Şen  Ayşen Guruda.mp3",
    "Nasıl Yani _ Türk Filmi  ( 2016 Full HD Tek Parça İzle ).mp3",
    "Musallat HD Burak Özcivit Türk Korku Filmi.mp3",
    "Musallat 2.mp3",
    "Musallat 1 Full izle _ Yerli Korku Filmi (Cin Filmleri).mp3",
    "Neredesin Firuze _ Türk Komedi Filmi Tek Parça (HD).mp3",
    "New York'ta Beş Minare HD (2010).mp3",
    "Neşeli Günler _ FULL HD.mp3",
    "Oldu mu Şimdi komedi filmi.mp3",
    "Olur Olur _ Türk Komedi Filmi Tek Parça (HD).mp3",
    "Orada _ Dram Filmi _ Full HD Tek Parça.mp3",
    "Organik Aşk Hikayeleri _ Romantik Türk Filmi.mp3",
    "Pardon _ Ferhan Şensoy, Rasim Öztekin, Bülent Kayabaş _ Tek Parça Komedi Filmi.mp3",
    "Peri Masalı _ Türk Filmi Tek Parça (HD).mp3",
    "Postacı _ Restorayonlu Kemal Sunal _ Fatma Girik Türk Filmi HD İzle.mp3",
    "Propaganda (1999).mp3",
    "Robinson Crusoe ve Cuma _ Türk Komedi Filmi.mp3",
    "Rüzgar  Yerli Film (Tek Parça) HD.mp3",
    "Sadece Sen (2013 _ HD) _ Türk Filmi.mp3",
    
    "Sarmaşık Gülleri _ Kartal Tibet Ve Hülya Koçyiğit  Eski Türk Filmi Tek Parça.mp3",
    "Sağ Salim _ Tek Parça Full HD (Yerli Film).mp3",
    "Selvi Boylum Al Yazmalım (1977) _ Türkan Şoray  Kadir İnanır  Ahmet Mekin (HD).mp3",
    "Sevimli Tehlikeli _ Tek Parça Film (Yerli Film).mp3",
    "Sicccin 8 yerli korku filmi 2020   FULL HD FİLM İZLE.mp3",
    "Siccin 4 (2017 _ Full HD).mp3",
    "Siccin 5 (2018 _ Full HD).mp3",
    "Siccin 6 (2019 _ Full HD).mp3",
    "Sihirbazlık Okulunda Bir Türk  Tek Parça  Full İzle.mp3",
    "Sir_Ayet 2 _ Türk Korku Filmi.mp3",
    "Son Mektup _ Türk Filmi Tek Parça (HD).mp3",
    "Son Urfalı _ Eski Türk Filmi Tek Parça.mp3",
    "Sosyete Şaban Türk Filmi _ FULL _ 4K ULTRA HD _ KEMAL SUNAL _ PERİHAN SAVAŞ.mp3",
    "Sosyetik Gelin Yerli Romantik Komedi Filmi HD İzle.mp3",
    "Su ve Ateş _ Tek Parça Film (Yerli Film).mp3",
    "Suveydâ yerli film izle 2021.mp3",
    "SİCCİN 1.mp3",
    
    "Sır Çocukları _ Film.mp3",
    "Taş Mektep _ Türk Dram Filmi.mp3",
    "Terkedilmiş _ Türk Dram Filmi İzle.mp3",
    "Tersine Dünya _ Eski Türk Filmi Tek Parça.mp3",
    "Tosun Paşa _ FULL HD.mp3",
    "Turk Filmi Tek Parça.mp3",
    "Tutku Türk Filmi _ FULL HD _ HÜLYA AVŞAR _ KENAN KALAV.mp3",
    "Tutmayın Beni _ Türk Komedi Filmi Tek Parça (HD).mp3",
    "Türk Filmi Tek Parça (HD).mp3",
    "Türk İşi Korku Filmi _ Vol 1_2_3_4_5_6_7_8 _ Tahsin Hasoğlu.mp3",
    "Türkler Çıldırmış Olmalı _ Tek Parça Film (Yerli Film).mp3",
    "Ulak _ Tek Parça Film (Yerli Film).mp3",
    "Umudun Son Kırıntısı (Dram) HD _ 2017 Türk Filmi.mp3",
    "Vali _ Sinema Filmi.mp3",
    "Vay Başıma Gelenler 2 Buçuk _ Türk Filmi Tek Parça (HD).mp3",
    "Ve Panayır Köyden Gider İzle Full HD Sansürsüz Yerli Film (2017).mp3",
    "Vezir Parmağı (2017 _ Full HD) _ Türk Filmi.mp3",
    "x2mate.com - Ammar 2 _ Cin İstilası _ Türk Korku Filmi Tek Parça (HD) (192 kbps).mp3",
    "x2mate.com - Ayla _ Savaşın kızı - izle tek parça (Oscar Adayı Film) yüklenme tarihi 2018 (128 kbps).mp3",
    "x2mate.com - Hannas_ Karanlıkta Saklanan FULL HD izle (Cin Filmleri) (192 kbps).mp3.",
    "x2mate.com - Kolpaçino_ Bomba _ Şafak Sezer Türk Filmi Full İzle (192 kbps).mp3",
    "x2mate.com - Babam ve Oğlum - Tek Parça Film (Yerli Film) (Yönetmen_ Çağan Irmak) (128 kbps).mp3",
    "x2mate.com - Hannas_ Karanlıkta Saklanan FULL HD izle (Cin Filmleri) (192 kbps).mp3",
    "x2mate.com - Helak_ Kayıp Köy _ 4K ULTRA HD Korku Filmi İzle (192 kbps).mp3",
    "x2mate.com - Maskeli Beşler _ İntikam Peşinde Tek Parça (HD) (192 kbps).mp3",
    "x2mate.com - Moskova'nın Şifresi_ Temel _ Türk Komedi Filmi Tek Parça (HD) (128 kbps).mp3",
    "x2mate.com - MUCİZE 2 (MUCİZE 2_AŞK) (128 kbps).mp3",
    "YA SONRA  ROMANTİK KOMEDİ YERLİ FİLM FULL TEK PART FİLM İZLE 2022 ( DİZİ FİLM İZLEMECE ).mp3",
    "Yahşi Batı _ Cem Yılmaz Türk Komedi Filmi _ Full Film İzle (HD).mp3",
    "Yalan Dolan _ Tek Parça HD İzle _ Yerli Komedi.mp3",
    "Yasaklı Köy Türk Filmi _ FULL _ Gerilim Korku Filmi.mp3",
    "Yav He He _ Türk Komedi Filmi Tek Parça (HD).mp3",
    "Yoksul Türk Filmi _ RESTORASYONLU _ Kemal Sunal Filmleri.mp3",
    "YOL _ TÜRK FİLMİ _ FULL HD _ TARIK AKAN _ HALİL ERGÜN _ ŞERİF SEZER.mp3",
    "Yola Çıkmak _ Yerli Romantik Dram Filmi.mp3",
    "Yüz Bin Kibrit _ Türk Filmi Tek Parça (HD).mp3",
    "Yüz Numaralı Adam _ Restorasyonlu Kemal Sunal Türk Komedi Filmi.mp3"
]

In [6]:
folder_path = "mp3_files"

if os.path.exists(folder_path):
    files = os.listdir(folder_path) # get list of files
    mp3_files = [file for file in files if file.lower().endswith(".mp3")] # get all files endswith '.mp3'
    for mp3 in mp3_files:
        if mp3 not in done:
            full_path = os.path.abspath(os.path.join(folder_path, mp3))
            print(f"Transcribing {mp3}")

            transcribe = model.transcribe(f"{full_path}", language="tr")
        
            print("Writing...")
            df = pd.DataFrame({"name": mp3, "transcript": [transcribe["text"]]})

            df.to_csv('transcript_files/transcriptions.csv', sep='|', mode="a", index=False)

Transcribing Zaman Makinesi 1973 _ Gürgen Öz, Seda Bakan.mp3
Writing...
Transcribing Zıkkımın Kökü _ Türk Filmi 1992.mp3
Writing...
Transcribing Çakal   türk filmleri 2013 tek parça 1.mp3
Writing...
Transcribing Çöpçüler Kralı _ FULL HD.mp3
Writing...
Transcribing Ölümlü Dünya _ Feyyaz Yiğit FULL HD Komedi Filmi İzle.mp3
Writing...
Transcribing Öz Hakiki Karakol _  Türk Filmi Tek Parça (HD).mp3
Writing...
Transcribing Öğretmen _ Kemal Sunal Komedi Filmi.mp3
Writing...
Transcribing Üç Kağıtçı Türk Filmi _ FULL HD _ Kemal Sunal Filmleri.mp3
Writing...
Transcribing İki Kafadar _ Türk Komedi Filmi Tek Parça (HD).mp3
Writing...
Transcribing İkimize Bir Dünya (Yerli Film) HD.mp3
Writing...
Transcribing İncir Çekirdeği _ Türk Filmi Dram Filmi _ Full Film İzle.mp3
Writing...
Transcribing İçimdeki Ses _ Türk Komedi Filmi (Tek Parça).mp3
Writing...
Transcribing Şark Bülbülü Türk Filmi _ RESTORASYONLU _ Kemal Sunal Filmleri.mp3
Writing...
Transcribing Şerr_i Cin Adlı Yerli Korku Filmi (2023).mp3


In [7]:
d = pd.read_csv('transcript_files/transcriptions.csv', sep='|')

,name,transcript
0,A.R.O.G _ Cem Yılmaz Türk Komedi Filmi.mp3,İzlediğiniz için teşekkür ederim. Altyazı M.K...
1,name,transcript
2,Abimm _ Türk Filmi Tek Parça (HD).mp3,Bu dizinin betimlemesi TRT tarafından Sesli B...
3,name,transcript
4,Acıların Kadını Bergen _ Yerli Film _ Weco Fil...,Yıllar yıllık dert yolunda Ne ilk neden sonun...


In [8]:
d.shape

(383, 2)

In [12]:
d[d["name"] != "name"].shape